# Train Normal Model

In [1]:
from cv2 import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import os.path

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers, models

from pprint import pprint as pp

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


2021-11-23 01:38:44.999202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 01:38:45.009061: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 01:38:45.009927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
from asl_model import *

TRAIN_DIR = 'DATASET_A_AP/train'
TEST_DIR = 'DATASET_A_AP/test'

BATCH = 128
EPOCH = 100

model_func = get_model_1

checkpoint_path = "ckpt_normal_training/cp.ckpt"
tb_log_dir = 'normal-logs/'
entire_model_s_path = 'saved_model/normal_model'

In [4]:
# Convert folder to dataframe of images' paths & labels
def get_paths_labels(path, allowed_extension="jpg"):
        global Path
        images_dir = Path(path)
        
        filepaths = pd.Series((images_dir.glob(fr'**/*.{allowed_extension}'))).astype(str)
        filepaths.name = "path"
        
        labels = filepaths.str.split("/")[:].str[-2]
        labels.name = "label"

        # Concatenate filepaths and labels
        df = pd.concat([filepaths, labels], axis=1)

        # Shuffle the DataFrame and reset index
        df = df.sample(frac=1).reset_index(drop = True)
        return df

In [5]:
train_df = get_paths_labels(TRAIN_DIR)
test_df = get_paths_labels(TEST_DIR)

In [6]:
data_generator = ImageDataGenerator(validation_split=0.2,
                                    rescale=1. / 255.,
                                    zoom_range=0.1,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    shear_range=0.1,)

test_generator = ImageDataGenerator(rescale=1. / 255.)

train_images = data_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=BATCH,
    shuffle=True,
    subset='training'
)

val_images = data_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=BATCH,
    shuffle=True,
    subset='validation'
)


test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    # batch_size=1,
    shuffle=False,
)

Found 46174 validated image filenames belonging to 26 classes.
Found 11543 validated image filenames belonging to 26 classes.
Found 14430 validated image filenames belonging to 26 classes.


# Callback Settings

In [7]:
# checkpoint_path = "ckpt_normal_training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=2, 
    save_weights_only=True,
    # save_best_only=True,
    save_freq=5*BATCH
)


es_callback = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau()


tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1)

callback_ls = [
    es_callback, 
    reduce_lr_callback, 
    cp_callback,
    tb_callback,
]

In [8]:
def create_model():
    model = model_func()
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [9]:
model = create_model()

model.fit(train_images, 
          validation_data=val_images,
          epochs=EPOCH,
          callbacks=callback_ls)

2021-11-23 01:38:46.456116: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-23 01:38:46.457104: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 01:38:46.457979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-23 01:38:46.458782: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/100


2021-11-23 01:38:48.071391: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204


361/361 [==============================] - 23s 58ms/step - loss: 0.5690 - accuracy: 0.8327 - val_loss: 2.1054 - val_accuracy: 0.3393 - lr: 0.0010
Epoch 2/100
278/361 [======================>.......] - ETA: 3s - loss: 0.1715 - accuracy: 0.9478
Epoch 00002: saving model to ckpt_normal_training/cp.ckpt
361/361 [==============================] - 21s 59ms/step - loss: 0.1646 - accuracy: 0.9498 - val_loss: 0.1477 - val_accuracy: 0.9548 - lr: 0.0010
Epoch 3/100
361/361 [==============================] - 21s 58ms/step - loss: 0.1098 - accuracy: 0.9660 - val_loss: 0.1196 - val_accuracy: 0.9654 - lr: 0.0010
Epoch 4/100
195/361 [===============>..............] - ETA: 7s - loss: 0.0834 - accuracy: 0.9749
Epoch 00004: saving model to ckpt_normal_training/cp.ckpt
361/361 [==============================] - 21s 58ms/step - loss: 0.0840 - accuracy: 0.9747 - val_loss: 0.1054 - val_accuracy: 0.9690 - lr: 0.0010
Epoch 5/100
361/361 [==============================] - 21s 58ms/step - loss: 0.0729 - accuracy

# save entire model

In [10]:
model.save(entire_model_s_path)

2021-11-23 01:43:47.540609: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: saved_model/normal_model/assets


# reload the entire model

In [11]:
new_model = tf.keras.models.load_model(entire_model_s_path)

In [12]:
new_model.evaluate(test_images)

451/451 [==============================] - 5s 10ms/step - loss: 0.5360 - accuracy: 0.8949


[0.5359690189361572, 0.8948717713356018]

# reload weight by checkpoint

In [13]:
new_model = create_model()

In [14]:
new_model.load_weights(checkpoint_path)

In [15]:
loss, acc = new_model.evaluate(test_images)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

451/451 [==============================] - 5s 9ms/step - loss: 0.6393 - accuracy: 0.8863
Restored model, accuracy: 88.63%
